In [ ]:
# Iporting necessary libraries
import time
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure = 'retina'
import random
from functools import reduce
!pip install spotipy --upgrade
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 13.5 MB/s eta 0:00:00


In [ ]:
# Insert your spotify username and the credentials from Spotify developer
cid = '/* you get this on SpotifyAPI*/'
secret = '****************'
redirect_uri='https://colab.research.google.com/drive/1042ZhLS-LTYdU9YJDv6qm_RYzsWW0CUt/callback'
username = '***********************'

In [ ]:
scope = 'user-top-read playlist-modify-private playlist-modify-public'
sp = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(
          client_id=cid,
          client_secret= secret,
          redirect_uri= redirect_uri,
          scope=scope, open_browser=False))
#token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

#if token:
#   sp = spotipy.Spotify(auth=token,open_browser=False)
#else:
#    print("Can't get token for", username)

print(sp.current_user())
print("\n")
print(sp)


Go to the following URL: https://accounts.spotify.com/authorize?client_id=27493d0fede849849d953414bc8a8314&response_type=code&redirect_uri=https%3A%2F%2Fcolab.research.google.com%2Fdrive%2F1042ZhLS-LTYdU9YJDv6qm_RYzsWW0CUt%2Fcallback&scope=user-top-read+playlist-modify-private+playlist-modify-public
Enter the URL you were redirected to: https://colab.research.google.com/drive/1042ZhLS-LTYdU9YJDv6qm_RYzsWW0CUt/callback?code=AQCkswL327O6rFO7GgnhYWlMQVA7VOZzDB2m6ZuX1ef-zjgM94ry0ooZorgg7w9LI20Pp9vR0oj5ryRUKLOG8ebBQdLP8BtDrDqc1vyZRiEqK4QfBuzCxVJmZ6tdCTzVjfRGsKVw9SqyD_d4DfRti0gKJNN6aBn_EZ7sIKJ7qrOLoVgqLtZJL6iugyQwxe7j9UhAuX2gplQp0lUjwHzOQrtiQWtSQu2xt-aJGS0VgHxRG0zVIuFHbYsrg6YYVSqKuejkLEycuLiZGo5HP2WpNVHjhEVYpfY0SO-r6fQ98uyCGSDjkYcZDGu2e0AqJc_-T6Ak34eR
{'display_name': 'Avom', 'external_urls': {'spotify': 'https://open.spotify.com/user/zmefuc6bodbfwnqvcjqxr4fua'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/users/zmefuc6bodbfwnqvcjqxr4fua', 'id': 'zmefuc6bodbf

# Creating the dataframe that will hold the track details.

In [ ]:
# Get features for songs
def fetch_audio_features(sp, df):
    playlist = df[['track_id','track_name']]
    index = 0
    audio_features = []

    #API Request
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50

    # create empty list to feed in track characteristics
    features_list = []
    #Create key-value of empty lists inside nested dictionary for album
    for features in audio_features:
        features_list.append([features['danceability'],
                             features['acousticness'],
                             features['energy'],
                             features['tempo'],
                             features['instrumentalness'],
                             features['loudness'],
                             features['liveness'],
                             features['duration_ms'],
                             features['key'],
                             features['valence'],
                             features['speechiness'],
                             features['mode']
                             ])

    df_audio_features = pd.DataFrame(features_list, columns=['danceability','acousticness','energy','tempo',
                                                            'instrumentalness', 'loudness','liveness','duration_ms',
                                                            'key','valence','speechiness','mode'])

    # Creating the final dataframe, with track_id as the index
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features

# **Getting Songs from your current playlist**
The initial plan was to get all songs from the official Spotify account. However, due to some API changes on Spotify's end. I had to change approach to getting a dataframe of around 750 songs. Which is really small for a machine learning dataset. I managed to get a dataset with 10,000 songs which is good for what I am trying to achieve.

In [ ]:
# Getting the playlist IDs from each of the user's playlists
playlists = sp.current_user_playlists()
spotify_playlist_ids = []
while playlists:
    for i, playlist in enumerate(playlists['items']):
        spotify_playlist_ids.append(playlist['uri'][-22:])
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
         playlists = None
spotify_playlist_ids[:20]

['3cpm63ERNW5ny8SacpB76v',
 '7KRh4PbNlf0q7Lc8ENijZb',
 '77gA1L9jSrMs0h2QqD3z6N',
 '0eKYCAfFLCGmb4YVzKdlh5',
 '2e62pfybi24aNR1kO5NlZj',
 '4bXoOoKVm16qOFo06duASU',
 '37i9dQZF1E36Rlu4WoVPlS',
 '37i9dQZF1DX3vcCtewyfc1',
 '37i9dQZF1DWXbzlSZRAZpg',
 '0bSDFeLjNjail4EMpRhU7x',
 '3tVeZ5avLyeTywr1ODyFO0',
 '0uMHrsCbX24pJ3sTxP7AWg',
 '5KrRcx8tejcwitWLQ2yLU9',
 '22gqkccpUKqKB4dyPZfpJe',
 '2IT0urjDzVDjiu1NxW7fRz',
 '639yWmetwwsOilkdHgYAnn',
 '2k6HSi0bmWLYOoTyixyT0f',
 '1zwFPYFRpvbG0tlJwlNkRc',
 '6kzAq30rrpWF3HshrYgAgr',
 '2jSOvE0ZdQCozLvtk8qKMl']

In [ ]:
len(spotify_playlist_ids)

22

In [ ]:
def getTrackIDs(playlist_id):
    playlist = sp.user_playlist('spotify', playlist_id)
    for item in playlist['tracks']['items'][:50]:
        track = item['track']
        ids.append(track['id'])
    return


# Getting the individual track details from the user's Spotify playlists

In [ ]:
#Function to get features of eacj track from track id
def getTrackFeatures(track_id):
    meta = sp.track(track_id,market='KE')
    features = sp.audio_features(track_id)

    # meta
    track_id = track_id
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    #features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']

    track = [track_id, name, album, artist, release_date, length, popularity, acousticness,
            danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    return track

The below cell takes sometime to run.(2 - 5 mins) on your PC.

In [ ]:
# Gather track ids
ids = []
for x in spotify_playlist_ids[:200]:
    getTrackIDs(x)
ids[:5]
print(len(ids))

705


The cell below depending on your PC.(If you have a GPU feel free to run it). Just use cloud computing(Google Colab, Anaconda Cloud).
It's almost 12000x faster.

In [ ]:
# Loop over track ids to get audio features for each track
tracks = []

for i in range(len(ids)):
    try:
        track = getTrackFeatures(ids[i])
        tracks.append(track)
    except:
        pass

#create a dataset
df = pd.DataFrame(tracks, columns = ['track_id', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability' ,
                                     'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df.head()

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,7vBEx9HgiEJDMwougw5bsa,Honey - Medasin Remix,Honey Remixed,KUČKA,2016-04-06,230958,56,0.722,0.447,0.715,0.000145,0.128,-6.616,0.4950,98.015,4
1,0Lgk4pD3Y0iE9w2xDilhjj,Spread Too Thin (feat. Lindsay Olsen),Manj,Joomanji,2013-02-01,285669,49,0.728,0.405,0.456,0.600000,0.375,-9.498,0.1570,87.721,4
2,7lPN2DXiMsVn7XUKtOW1CS,drivers license,drivers license,Olivia Rodrigo,2021-01-08,242013,88,0.721,0.585,0.436,0.000013,0.105,-8.761,0.0601,143.874,4
3,02nfBodSDuFx0Wp7XV4bn8,Moonshine,3,Ngaiire,2021-08-27,208564,16,0.561,0.771,0.576,0.000116,0.226,-6.875,0.0391,112.993,4
4,0BQW5N0eR3GAJtMBwScrl7,Can't We,Mantra,Tyde,2021-04-30,235213,18,0.762,0.751,0.387,0.853000,0.105,-11.283,0.1130,84.025,4


In [ ]:
df.to_csv('playlist_songs.csv', index=False)

In [ ]:
df = pd.read_csv('playlist_songs.csv')
df.head()

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,7vBEx9HgiEJDMwougw5bsa,Honey - Medasin Remix,Honey Remixed,KUČKA,2016-04-06,230958,56,0.722,0.447,0.715,0.000145,0.128,-6.616,0.4950,98.015,4
1,0Lgk4pD3Y0iE9w2xDilhjj,Spread Too Thin (feat. Lindsay Olsen),Manj,Joomanji,2013-02-01,285669,49,0.728,0.405,0.456,0.600000,0.375,-9.498,0.1570,87.721,4
2,7lPN2DXiMsVn7XUKtOW1CS,drivers license,drivers license,Olivia Rodrigo,2021-01-08,242013,88,0.721,0.585,0.436,0.000013,0.105,-8.761,0.0601,143.874,4
3,02nfBodSDuFx0Wp7XV4bn8,Moonshine,3,Ngaiire,2021-08-27,208564,16,0.561,0.771,0.576,0.000116,0.226,-6.875,0.0391,112.993,4
4,0BQW5N0eR3GAJtMBwScrl7,Can't We,Mantra,Tyde,2021-04-30,235213,18,0.762,0.751,0.387,0.853000,0.105,-11.283,0.1130,84.025,4


In [ ]:
# Dropping columns that could lead to data leakage
df = df.drop(columns=['name', 'album', 'artist', 'release_date'])
df.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,7vBEx9HgiEJDMwougw5bsa,230958,56,0.722,0.447,0.715,0.000145,0.128,-6.616,0.4950,98.015,4
1,0Lgk4pD3Y0iE9w2xDilhjj,285669,49,0.728,0.405,0.456,0.600000,0.375,-9.498,0.1570,87.721,4
2,7lPN2DXiMsVn7XUKtOW1CS,242013,88,0.721,0.585,0.436,0.000013,0.105,-8.761,0.0601,143.874,4
3,02nfBodSDuFx0Wp7XV4bn8,208564,16,0.561,0.771,0.576,0.000116,0.226,-6.875,0.0391,112.993,4
4,0BQW5N0eR3GAJtMBwScrl7,235213,18,0.762,0.751,0.387,0.853000,0.105,-11.283,0.1130,84.025,4


# Getting the user's favorite tracks.
You'll get around 50 tracks. You can switch the time range to see how different create different recommendations.
(short_term, medium_term, long_term)

In [ ]:
# Getting the users top tracks
results = sp.current_user_top_tracks(limit=1000, offset=0, time_range='medium_term')

In [ ]:
# Convert the above results to a Dataframe
track_name = []
track_id = []
artist = []
album = []
duration = []
popularity = []
for i, items in enumerate(results['items']):
    track_name.append(items['name'])
    track_id.append(items['id'])
    artist.append(items["artists"][0]["name"])
    duration.append(items["duration_ms"])
    album.append(items["album"]["name"])
    popularity.append(items["popularity"])

# Creating the final dataframe
df_favourite = pd.DataFrame({ "track_name": track_name,
                             "album": album,
                             "track_id": track_id,
                             "artist": artist,
                             "duration": duration,
                             "popularity": popularity})
df_favourite.head()

,track_name,album,track_id,artist,duration,popularity
0,Crew (feat. Brent Faiyaz & Shy Glizzy),At What Cost,15EPc80XuFrb2LmOzGjuRg,GoldLink,176702,79
1,THIS SH!T,THIS SH!T,0NfxQ1dLKqwT40AhR6zqAY,AYLØ,170217,48
2,Two Tens (feat. Anderson .Paak),Two Tens (feat. Anderson .Paak),6clDsO8HwhHEgJDDp88VdL,Cordae,152371,75
3,Headshots (4r da Locals),The House Is Burning,0xta6Gah3inmeHbj0e3F9a,Isaiah Rashad,193208,69
4,Lock It Up,Minus The Bullshit Life's Great,1GIKDc28Z95bWSIXgkr3y3,Nascent,226865,53


In [ ]:
%time
# Getting track features for each song in favourite song dataframe
fav_tracks = []
for track in df_favourite['track_id']:
    try:
        track = getTrackFeatures(track)
        fav_tracks.append(track)
    except:
        pass


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


In [ ]:
# Create favourite track with audio features dataset
df_fav = pd.DataFrame(fav_tracks, columns = ['track_id', 'name', 'album','artist', 'release_date',
                                             'length', 'popularity', 'danceability','acousticness','energy','instrumentalness',
                                             'liveness','loudness','speechiness','tempo','time_signature'])
df_fav.head()

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,15EPc80XuFrb2LmOzGjuRg,Crew (feat. Brent Faiyaz & Shy Glizzy),At What Cost,GoldLink,2017-03-24,176702,79,0.1330,0.728,0.607,0.000000,0.0864,-8.417,0.1320,129.938,4
1,0NfxQ1dLKqwT40AhR6zqAY,THIS SH!T,THIS SH!T,AYLØ,2020-05-01,170217,48,0.6570,0.668,0.613,0.005310,0.1190,-7.623,0.3050,183.859,4
2,6clDsO8HwhHEgJDDp88VdL,Two Tens (feat. Anderson .Paak),Two Tens (feat. Anderson .Paak),Cordae,2023-01-25,152371,75,0.1940,0.603,0.546,0.000003,0.0783,-7.525,0.3500,87.990,4
3,0xta6Gah3inmeHbj0e3F9a,Headshots (4r da Locals),The House Is Burning,Isaiah Rashad,2021-07-30,193208,69,0.0308,0.894,0.615,0.009210,0.3090,-6.269,0.0826,120.000,4
4,1GIKDc28Z95bWSIXgkr3y3,Lock It Up,Minus The Bullshit Life's Great,Nascent,2021-05-14,226865,53,0.4190,0.661,0.587,0.000007,0.1080,-9.929,0.1970,119.934,4


In [ ]:
# Dropping columns that could lead to data leakage
df_fav = df_fav.drop(columns=['name', 'album', 'artist', 'release_date'])
df_fav.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,15EPc80XuFrb2LmOzGjuRg,176702,79,0.1330,0.728,0.607,0.000000,0.0864,-8.417,0.1320,129.938,4
1,0NfxQ1dLKqwT40AhR6zqAY,170217,48,0.6570,0.668,0.613,0.005310,0.1190,-7.623,0.3050,183.859,4
2,6clDsO8HwhHEgJDDp88VdL,152371,75,0.1940,0.603,0.546,0.000003,0.0783,-7.525,0.3500,87.990,4
3,0xta6Gah3inmeHbj0e3F9a,193208,69,0.0308,0.894,0.615,0.009210,0.3090,-6.269,0.0826,120.000,4
4,1GIKDc28Z95bWSIXgkr3y3,226865,53,0.4190,0.661,0.587,0.000007,0.1080,-9.929,0.1970,119.934,4


In [ ]:
# Checking for duplicates in df_fav
df_fav['track_id'].value_counts()

15EPc80XuFrb2LmOzGjuRg    1
1ePHqUkosKq7ZgQMz655Cw    1
4mD9hjoTckjeL1CrJCx0MT    1
5UHGymgm6Okr0oSQOuFNiF    1
6w2WWyBAOXQeP6waVT9s5O    1
6Nc7K0edFOawc4JByfAoPd    1
02aJMRtaJUSowdbjhmixXs    1
0J5FGPgIiL0ieCN06CEI4h    1
4IOYk6epkzXT38U4ShauZ6    1
5cRCEOCVmqUBRYW8XyFk0d    1
3yu5otkADG1ldufrPxABoo    1
2e8OPXNNk9onHe4pknBZhD    1
2epLYWuz0QLII6EOmLxkDj    1
0NfxQ1dLKqwT40AhR6zqAY    1
4x8874idDYp8yxgKsyb4xG    1
7cLnbmx7ZUWAwyocrKz3wL    1
6m3GdioadMVmcwRUlH8Sbc    1
2w9htZtkcbR48NCWzXjfvb    1
1bb6bROP3OXMHcPw84MtmN    1
2GhxLGEBrIjMFv5yDD1dEi    1
3XBkrymCZ2nfqgB5TjJJCI    1
67sHZvyNVEP6JIpiCJznsZ    1
2sVcw1xyQe44BGnEKLTzJB    1
078lbDGGTqBuTKU3bypXxQ    1
1gMT8boiNdGWiYy7HQHyZk    1
6DXEZzM4wBa3EOl0NCNSvz    1
1BSZGNpIm2nBt1hWSK2bRp    1
6v9ZXjyI0G6cK60I5y3t7x    1
6clDsO8HwhHEgJDDp88VdL    1
0xta6Gah3inmeHbj0e3F9a    1
1GIKDc28Z95bWSIXgkr3y3    1
3KZE0uFYL5DdReSh3POygX    1
7335Fg82PxOcKB1tWYdXHl    1
3sTN90bIP2cJ1783ctHykO    1
54I1dC6Ux5I4h7U44DmjgK    1
0ZBchpb4CLk5Qqu3895E

In [ ]:
# Creating favourite column to use in classification
df_fav['favorite'] = 1
df['favorite'] = 0

In [ ]:
# Checking if both datasets have the same columns
df.columns == df_fav.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

# Preparing dataset for model creation

In [ ]:
#Preparing dataset for model creation
print(df.shape)
print(df_fav.shape)

(704, 13)
(50, 13)


In [ ]:
# Combining favourite dataframe with playlists songs
combined = pd.concat([df, df_fav])
combined.shape

(754, 13)

In [ ]:
combined.favorite.value_counts()

0    704
1     50
Name: favorite, dtype: int64

The favorite songs to not playlist songs is imbalanced, will need to be fixed when building the model.

In [ ]:
df_fav = combined.loc[combined['favorite'] == 1]
df_fav.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,favorite
0,15EPc80XuFrb2LmOzGjuRg,176702,79,0.1330,0.728,0.607,0.000000,0.0864,-8.417,0.1320,129.938,4,1
1,0NfxQ1dLKqwT40AhR6zqAY,170217,48,0.6570,0.668,0.613,0.005310,0.1190,-7.623,0.3050,183.859,4,1
2,6clDsO8HwhHEgJDDp88VdL,152371,75,0.1940,0.603,0.546,0.000003,0.0783,-7.525,0.3500,87.990,4,1
3,0xta6Gah3inmeHbj0e3F9a,193208,69,0.0308,0.894,0.615,0.009210,0.3090,-6.269,0.0826,120.000,4,1
4,1GIKDc28Z95bWSIXgkr3y3,226865,53,0.4190,0.661,0.587,0.000007,0.1080,-9.929,0.1970,119.934,4,1


In [ ]:
df = combined.loc[combined['favorite'] != 1]
df.shape

(704, 13)

In [ ]:
df_fav.shape

(50, 13)

In [ ]:
# Saving these dataframes to use in model creation
df.to_csv('encoded_playlist_songs.csv', index=False)
df_fav.to_csv('favorite_songs.csv', index=False)